In [2]:
!pip install torch torchvision

   ---------------------------------------- 0.0/216.1 MB ? eta -:--:--
   ---------------------------------------- 0.3/216.1 MB ? eta -:--:--
   ---------------------------------------- 1.8/216.1 MB 6.0 MB/s eta 0:00:36
    --------------------------------------- 4.5/216.1 MB 8.8 MB/s eta 0:00:24
   - -------------------------------------- 7.1/216.1 MB 10.0 MB/s eta 0:00:21
   - -------------------------------------- 9.4/216.1 MB 10.6 MB/s eta 0:00:20
   -- ------------------------------------- 11.5/216.1 MB 10.8 MB/s eta 0:00:19
   -- ------------------------------------- 14.2/216.1 MB 10.7 MB/s eta 0:00:19
   --- ------------------------------------ 16.8/216.1 MB 11.0 MB/s eta 0:00:19
   --- ------------------------------------ 19.4/216.1 MB 11.2 MB/s eta 0:00:18
   --- ------------------------------------ 21.2/216.1 MB 11.0 MB/s eta 0:00:18
   ---- ----------------------------------- 23.9/216.1 MB 11.2 MB/s eta 0:00:18
   ---- ----------------------------------- 26.5/216.1 MB 11.3 M

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim


In [4]:
x = torch.tensor([[1.0], [2.0], [3.0], [4.0]])
y = torch.tensor([[0.0], [0.0], [1.0], [1.0]])
model = nn.Linear(in_features=1, out_features=1)
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [5]:
for epoch in range(100):
    y_pred = model(x)
    loss = loss_fn(y_pred, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

Epoch 0, Loss: 1.2894
Epoch 10, Loss: 0.7462
Epoch 20, Loss: 0.7072
Epoch 30, Loss: 0.6770
Epoch 40, Loss: 0.6489
Epoch 50, Loss: 0.6227
Epoch 60, Loss: 0.5982
Epoch 70, Loss: 0.5754
Epoch 80, Loss: 0.5542
Epoch 90, Loss: 0.5344


In [6]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_data = datasets.MNIST(root='data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=1000)

100%|██████████████████████████████████████████████████████████████████████████████| 9.91M/9.91M [00:10<00:00, 966kB/s]
100%|█████████████████████████████████████████████████████████████████████████████| 28.9k/28.9k [00:00<00:00, 90.1kB/s]
100%|██████████████████████████████████████████████████████████████████████████████| 1.65M/1.65M [00:07<00:00, 221kB/s]
100%|██████████████████████████████████████████████████████████████████████████████| 4.54k/4.54k [00:00<00:00, 875kB/s]


In [7]:
class FeedforwardNN(nn.Module):
    def __init__(self):
        super(FeedforwardNN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.fc3(x)

In [10]:
model = FeedforwardNN()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
for epoch in range(5):
    model.train()
    total_loss = 0

    for images, labels in train_loader:
        outputs = model(images)
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

Epoch 1, Loss: 257.1964
Epoch 2, Loss: 105.2480
Epoch 3, Loss: 75.6986
Epoch 4, Loss: 58.8634
Epoch 5, Loss: 47.8666


In [9]:
correct = 0
total = 0

model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Test Accuracy: 97.63%
